In [1]:
from perception import realsense_sensor
from datetime import datetime
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
from os.path import join
import pickle as pkl
import pyrealsense2 as rs
import time
from time import sleep

Using matplotlib backend: <object object at 0x7f443c457540>


In [2]:
# This script allows recording and playing back realsense data: both rbg and depth.

RECORD = False
# RECORD = True
PLAYBACK = True
# PLAYBACK = False

realsense_serial_num = '036522071871'

# Record realsense video

In [3]:
if RECORD:
    print("Recording realsense data")
    output_dir = "../data/household_shelves"
    freq = 5
    video_length = 15
    
    # list of color and dpeth images
    recording = {'data': [],
                 'freq': freq,
                 'video_length': video_length}

    # create realsense sensor (check the serial # on the device)
    sensor = realsense_sensor.RealSenseSensor(realsense_serial_num)
    
    filename =  'realsense_data_' + datetime.now().strftime("%d-%m-%Y-%H-%M-%S") + ".pkl"
    start_time = time.time()
    try:
        print("Starting realsense")
        sensor.start()
        while time.time() - start_time < video_length:
            color_img, depth_img = sensor.frames()
            if not depth_img: continue
            if not color_img: continue
            # depth_mat = np.array(depth.get_data())
            # color_mat = np.array(color.get_data())
            recording['data'].append({
                'rgb': color_img,
                'depth': depth_img})
    
            sleep(1.0/5)
            
        pkl.dump(recording, open(join(output_dir, filename), 'wb'))
        print("Recording saved to ", join(output_dir, filename))
    finally:
        sensor.stop()


# Play recorded realsense video

In [4]:
if PLAYBACK:
    filename = join("../data/household_shelves", "realsense_data_25-10-2023-18-45-23.pkl")
    recording = pkl.load(open(filename, 'rb'))
    data = recording['data']
    
    print("Playing recording ", filename, "of length ", recording['video_length'], "recorded with freq ", recording['freq'])
    
    plt.ion()
    fig, axs = plt.subplots(2)
    
    for frame in data:
        axs[0].imshow(frame['rgb'].data)
        axs[1].imshow(frame['depth'].data)
        plt.pause(1.0/recording['freq'])

Playing recording  ../data/household_shelves/realsense_data_25-10-2023-18-45-23.pkl of length  15 recorded with freq  5
